In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

In [2]:
train = pd.read_csv('train.csv')
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [3]:
train = train.drop(['ID', 'Var_1'], axis = 1)

In [4]:
train.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Segmentation
0,Male,No,22,No,Healthcare,1.0,Low,4.0,D
1,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,A
2,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,B
3,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,B
4,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,A


In [5]:
train.describe()

,Age,Work_Experience,Family_Size
count,8068.000000,7239.000000,7733.000000
mean,43.466906,2.641663,2.850123
std,16.711696,3.406763,1.531413
min,18.000000,0.000000,1.000000
25%,30.000000,0.000000,2.000000
50%,40.000000,1.000000,3.000000
75%,53.000000,4.000000,4.000000
max,89.000000,14.000000,9.000000


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           8068 non-null   object 
 1   Ever_Married     7928 non-null   object 
 2   Age              8068 non-null   int64  
 3   Graduated        7990 non-null   object 
 4   Profession       7944 non-null   object 
 5   Work_Experience  7239 non-null   float64
 6   Spending_Score   8068 non-null   object 
 7   Family_Size      7733 non-null   float64
 8   Segmentation     8068 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 567.4+ KB


In [7]:
train.Ever_Married.unique()

array(['No', 'Yes', nan], dtype=object)

In [8]:
train.Ever_Married.value_counts()

Yes    4643
No     3285
Name: Ever_Married, dtype: int64

In [9]:
train['Graduated'].unique()

array(['No', 'Yes', nan], dtype=object)

In [10]:
train['Graduated'].value_counts()

Yes    4968
No     3022
Name: Graduated, dtype: int64

In [11]:
train['Graduated'].isnull().sum() / train.shape[0] * 100

0.9667823500247893

In [12]:
train['Ever_Married'].isnull().sum() / train.shape[0] * 100

1.7352503718393655

In [13]:
total_entries = train.shape[0]
total_entries

8068

In [14]:
train.isnull().sum()

Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Segmentation         0
dtype: int64

In [15]:
train['Family_Size'].value_counts()

2.0    2390
3.0    1497
1.0    1453
4.0    1379
5.0     612
6.0     212
7.0      96
8.0      50
9.0      44
Name: Family_Size, dtype: int64

In [16]:
import numpy as np
train['Family_Size'].replace(np.nan, 1, inplace = True)
train['Family_Size'].value_counts()

2.0    2390
1.0    1788
3.0    1497
4.0    1379
5.0     612
6.0     212
7.0      96
8.0      50
9.0      44
Name: Family_Size, dtype: int64

In [17]:
train['Work_Experience'].replace(np.nan, 0, inplace = True)
train.isnull().sum()

Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience      0
Spending_Score       0
Family_Size          0
Segmentation         0
dtype: int64

In [18]:
train['Profession'].replace(np.nan, 'Not stated', inplace = True)

In [19]:
train.dtypes

Gender              object
Ever_Married        object
Age                  int64
Graduated           object
Profession          object
Work_Experience    float64
Spending_Score      object
Family_Size        float64
Segmentation        object
dtype: object

In [20]:
train['Gender'] = train['Gender'].astype('str')
train['Ever_Married'] = train['Ever_Married'].astype('str')
train['Graduated'] = train['Graduated'].astype('str')

In [21]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
train['gender'] = enc.fit_transform(train['Gender'])
train['ever_married'] = enc.fit_transform(train['Ever_Married'])
train['graduated'] = enc.fit_transform(train['Graduated'])
train.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Segmentation,gender,ever_married,graduated
0,Male,No,22,No,Healthcare,1.0,Low,4.0,D,1,0,0
1,Female,Yes,38,Yes,Engineer,0.0,Average,3.0,A,0,1,1
2,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,B,0,1,1
3,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,B,1,1,1
4,Female,Yes,40,Yes,Entertainment,0.0,High,6.0,A,0,1,1


In [22]:
df = train.drop(['Gender', 'Ever_Married', 'Graduated'], axis = 1)
df.head()

,Age,Profession,Work_Experience,Spending_Score,Family_Size,Segmentation,gender,ever_married,graduated
0,22,Healthcare,1.0,Low,4.0,D,1,0,0
1,38,Engineer,0.0,Average,3.0,A,0,1,1
2,67,Engineer,1.0,Low,1.0,B,0,1,1
3,67,Lawyer,0.0,High,2.0,B,1,1,1
4,40,Entertainment,0.0,High,6.0,A,0,1,1


In [23]:
mapping = {'Low': 0, 'Average': 1, 'High': 2}
df['Spending_Score'] = df['Spending_Score'].map(mapping)
df.head()

,Age,Profession,Work_Experience,Spending_Score,Family_Size,Segmentation,gender,ever_married,graduated
0,22,Healthcare,1.0,0,4.0,D,1,0,0
1,38,Engineer,0.0,1,3.0,A,0,1,1
2,67,Engineer,1.0,0,1.0,B,0,1,1
3,67,Lawyer,0.0,2,2.0,B,1,1,1
4,40,Entertainment,0.0,2,6.0,A,0,1,1


In [24]:
small_df = pd.get_dummies(df['Profession'])
small_df

,Artist,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,Lawyer,Marketing,Not stated
0,0,0,0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
8063,0,0,0,0,0,0,0,0,0,1
8064,0,0,0,0,1,0,0,0,0,0
8065,0,0,0,0,0,1,0,0,0,0
8066,0,0,0,0,0,1,0,0,0,0


In [25]:
df = pd.concat([df, small_df], axis = 1)
df

,Age,Profession,Work_Experience,Spending_Score,Family_Size,Segmentation,gender,ever_married,graduated,Artist,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,Lawyer,Marketing,Not stated
0,22,Healthcare,1.0,0,4.0,D,1,0,0,0,0,0,0,0,1,0,0,0,0
1,38,Engineer,0.0,1,3.0,A,0,1,1,0,0,1,0,0,0,0,0,0,0
2,67,Engineer,1.0,0,1.0,B,0,1,1,0,0,1,0,0,0,0,0,0,0
3,67,Lawyer,0.0,2,2.0,B,1,1,1,0,0,0,0,0,0,0,1,0,0
4,40,Entertainment,0.0,2,6.0,A,0,1,1,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8063,22,Not stated,0.0,0,7.0,D,1,0,0,0,0,0,0,0,0,0,0,0,1
8064,35,Executive,3.0,0,4.0,D,1,0,0,0,0,0,0,1,0,0,0,0,0
8065,33,Healthcare,1.0,0,1.0,D,0,0,1,0,0,0,0,0,1,0,0,0,0
8066,27,Healthcare,1.0,0,4.0,B,0,0,1,0,0,0,0,0,1,0,0,0,0


In [26]:
df.drop(['Profession'], axis = 1, inplace = True)


In [27]:
df.head()

,Age,Work_Experience,Spending_Score,Family_Size,Segmentation,gender,ever_married,graduated,Artist,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,Lawyer,Marketing,Not stated
0,22,1.0,0,4.0,D,1,0,0,0,0,0,0,0,1,0,0,0,0
1,38,0.0,1,3.0,A,0,1,1,0,0,1,0,0,0,0,0,0,0
2,67,1.0,0,1.0,B,0,1,1,0,0,1,0,0,0,0,0,0,0
3,67,0.0,2,2.0,B,1,1,1,0,0,0,0,0,0,0,1,0,0
4,40,0.0,2,6.0,A,0,1,1,0,0,0,1,0,0,0,0,0,0


In [28]:
df.shape

(8068, 18)

In [29]:
X, y = df.drop('Segmentation', axis = 1), df['Segmentation']
X.head()

,Age,Work_Experience,Spending_Score,Family_Size,gender,ever_married,graduated,Artist,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,Lawyer,Marketing,Not stated
0,22,1.0,0,4.0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,38,0.0,1,3.0,0,1,1,0,0,1,0,0,0,0,0,0,0
2,67,1.0,0,1.0,0,1,1,0,0,1,0,0,0,0,0,0,0
3,67,0.0,2,2.0,1,1,1,0,0,0,0,0,0,0,1,0,0
4,40,0.0,2,6.0,0,1,1,0,0,0,1,0,0,0,0,0,0


In [ ]:
y

In [ ]:
# scaler = MinMaxScaler()
# scaled_df = scaler.fit_transform(X)
# scaled_df[:10]

In [ ]:
scaler = MinMaxScaler()
X['scaled_age'] = scaler.fit_transform(X['Age'].values.reshape(-1,1))
# X['scaled_Work_Experience'] = scaler.fit_transform(X['Work_Experience'])
# X['scaled_Family_size'] = scaler.fit_transform(X['Family_size'])
X.head()                                              

In [ ]:
X['scaled_age'].max()
X['Age'].max()

In [ ]:
X['scaled_Work_Experience'] = scaler.fit_transform(X['Work_Experience'].values.reshape(-1,1))
X['scaled_Family_size'] = scaler.fit_transform(X['Family_Size'].values.reshape(-1,1))

In [ ]:
X.head()

In [ ]:
X.drop(['Age', 'Work_Experience','Family_Size'], axis = 1, inplace = True)

In [30]:
X.head()

,Age,Work_Experience,Spending_Score,Family_Size,gender,ever_married,graduated,Artist,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,Lawyer,Marketing,Not stated
0,22,1.0,0,4.0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,38,0.0,1,3.0,0,1,1,0,0,1,0,0,0,0,0,0,0
2,67,1.0,0,1.0,0,1,1,0,0,1,0,0,0,0,0,0,0
3,67,0.0,2,2.0,1,1,1,0,0,0,0,0,0,0,1,0,0
4,40,0.0,2,6.0,0,1,1,0,0,0,1,0,0,0,0,0,0


In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [32]:
knn = KNeighborsClassifier(n_neighbors = 4)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [33]:
y_pred = knn.predict(X_test)

In [34]:
y_pred

array(['D', 'A', 'B', ..., 'C', 'D', 'A'], dtype=object)

In [35]:
from sklearn.metrics import accuracy_score

In [36]:
print(accuracy_score(y_test, y_pred))

0.43579573624194345


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))